<a href="https://colab.research.google.com/github/libemurg1/Linux_Programming/blob/main/Copy_of_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
''' import libraries '''
import pandas as pd


In [2]:
''' load datasets into a dataframe'''
# load datasets into a a data frame
logs = pd.read_csv('/content/drive/MyDrive/Challenge/logs.csv')

# opening the scores dataset
scores = pd.read_csv('/content/drive/MyDrive/Challenge/scores.csv')

In [4]:
# view the first 10 entries
logs.head()

,StudentId,Time,Type,Action
0,72af,"28/05/23, 10:51",User report,Grade user report viewed
1,72af,"28/05/23, 10:51",System,Course viewed
2,c426,"27/05/23, 15:53",System,Course viewed
3,0326,"26/05/23, 22:22",System,Course viewed
4,8b7a,"26/05/23, 21:52",System,Course viewed


In [5]:
# view first 10 entries
scores.head()

,StudentId,Grade
0,c426,2nd
1,8de3,2nd
2,d969,2nd
3,6d29,1st
4,1dd9,1st


In [6]:
''' data cleaning - missing values handling '''
# drop entries with missing data
logs.dropna(inplace=True)

# drop entries with missing data
scores.dropna(inplace=True)


In [21]:
# Merge the logs and scores datasets based on the common 'StudentId' column
merged_data = pd.merge(logs, scores, on='StudentId')

# print merged data head
merged_data.head(n=10)

,StudentId,Time,Type,Action,Grade
0,72af,"28/05/23, 10:51",User report,Grade user report viewed,1st
1,72af,"28/05/23, 10:51",System,Course viewed,1st
2,c426,"27/05/23, 15:53",System,Course viewed,2nd
3,0326,"26/05/23, 22:22",System,Course viewed,1st
4,8b7a,"26/05/23, 21:52",System,Course viewed,Fail
5,8b7a,"26/05/23, 21:52",Open Grader,Open Grader viewed,Fail
6,8b7a,"26/05/23, 21:52",System,Course viewed,Fail
7,bde7,"26/05/23, 20:06",System,Course viewed,2nd
8,bde7,"26/05/23, 20:00",System,Course viewed,2nd
9,72af,"26/05/23, 09:58",User report,Grade user report viewed,1st


In [41]:
'''
Analyze the types of actions (in Type and Action columns) and how they might correlate with final grades.
'''
# Convert 'Grade' column to numeric representation if needed
# Create a mapping for grade values to numeric equivalents
grade_mapping = {
    '1st': 3,
    '2nd': 2,
    '3rd': 1,
    'Fail': 0  # You can assign a numeric value to 'Fail' based on your interpretation
}

# Apply the mapping to the 'Grade' column
merged_data['Grade'] = merged_data['Grade'].map(grade_mapping)


# Group by action type and calculate the average score for each type
action_type_scores = merged_data.groupby(['Type', 'Action'])['Grade'].mean()

# Print the results
print(action_type_scores)


Type                    Action                                        
Assignment              A submission has been submitted.                 NaN
                        Course module viewed                             NaN
                        Remove submission confirmation viewed.           NaN
                        Submission form viewed.                          NaN
                        The status of the submission has been updated.   NaN
                        The status of the submission has been viewed.    NaN
File                    Course module viewed                             NaN
File submissions        A file has been uploaded.                        NaN
                        Submission created.                              NaN
                        Submission updated.                              NaN
Folder                  Course module viewed                             NaN
                        Zip archive of folder downloaded                 NaN
Forum

Check for the correlation between grades and the below for feature engineering.
 1.  Assignment, The status of the submission has been updated and the grade.
 2. Questionnaire, Individual Responses report viewed.
 3. We can say anything that have an average above 1.5 on the above analysis.

In [23]:
'''
From the analysis above Assignment and Questionnaire have high mean we can add more features
That will be used by the model
'''
# Total interactions: Total number of actions a student took.
# Engagement in Assignment: Frequency of assignments actions.
# Engagement with Questionnaire: Frequency of questionnaire actions.

# Feature Engineering

# aggregations to create meaningful features
features = merged_data.groupby('StudentId').agg(
    total_actions=('Action', 'count'),
    assignment_actions=('Action', lambda x: (x == 'The status of the submission has been updated.').sum()),
    questionnaire_actions=('Type', lambda x: (x == 'Questionnaire').sum()),
    forum_actions=('Type', lambda x: (x == 'Forum').sum()),
    turnitin_assignment_two_actions=('Type', lambda x: (x == 'Turnitin Assignment 2').sum())
).reset_index()

features


,StudentId,total_actions,assignment_actions,questionnaire_actions,forum_actions,turnitin_assignment_two_actions
0,0126,126,0,0,0,4
1,0139,215,0,0,0,10
2,020c,813,0,0,57,11
3,026e,400,0,0,6,27
4,0326,1565,12,0,23,108
...,...,...,...,...,...,...
100,f63c,146,0,0,0,5
101,fb47,240,0,0,1,10
102,fc66,149,2,0,0,10
103,fd73,675,0,0,7,11


In [24]:
# merge features with grades to create a new dataset based on features and associted grades for the studentID
data = pd.merge(features, scores, on='StudentId')

# print data
data

,StudentId,total_actions,assignment_actions,questionnaire_actions,forum_actions,turnitin_assignment_two_actions,Grade
0,0126,126,0,0,0,4,2nd
1,0139,215,0,0,0,10,3rd
2,020c,813,0,0,57,11,Fail
3,026e,400,0,0,6,27,3rd
4,0326,1565,12,0,23,108,1st
...,...,...,...,...,...,...,...
100,f63c,146,0,0,0,5,1st
101,fb47,240,0,0,1,10,1st
102,fc66,149,2,0,0,10,2nd
103,fd73,675,0,0,7,11,Fail


Since the grades are categorical, encode them into numeric labels using LabelEncoder from scikit-learn.

In [25]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Grade'] = le.fit_transform(data['Grade'])


Split the data into training and test sets. Set aside some data for final testing (e.g., 80% for training and 20% for testing).

In [27]:
from sklearn.model_selection import train_test_split

# the training data without student id and grades
X = data.drop(columns=['StudentId', 'Grade'])

# the target column
y = data['Grade']

# split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Since this is a classification problem, you can choose models like Random Forest, Decision Tree, or Gradient Boosting. Here, we’ll use a Random Forest Classifier.

In [29]:
from sklearn.ensemble import RandomForestClassifier

# model to use
model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

Assess the model’s accuracy and classification report (precision, recall, F1-score) on the test set.

In [38]:
''' Import tools '''
from sklearn.metrics import accuracy_score, classification_report

# predict using model
y_pred = model.predict(X_test)

# print model accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# print classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.3333333333333333
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       0.25      0.20      0.22         5
           2       0.27      0.38      0.32         8
           3       0.50      0.17      0.25         6

    accuracy                           0.33        21
   macro avg       0.38      0.44      0.36        21
weighted avg       0.35      0.33      0.31        21



Load the unseen dataset (unseen.csv), preprocess it similarly (create features from logs as before), and use the trained model to predict grades.

In [40]:
# open the unseen dataset
unseen_data = pd.read_csv('/content/drive/MyDrive/Challenge/unseen.csv')

# Perform feature engineering on unseen_data
unseen_features = unseen_data.groupby('StudentId').agg(
    total_actions=('Action', 'count'),
    assignment_actions=('Type', lambda x: (x == 'Assignment').sum()),
    questionnaire_actions=('Type', lambda x: (x == 'Questionnaire').sum()),
    forum_actions=('Type', lambda x: (x == 'Forum').sum()),
    turnitin_assignment_two_actions=('Type', lambda x: (x == 'Turnitin Assignment 2').sum())
).reset_index()

# data to be predicted
X=unseen_features.drop(columns=['StudentId'])

# predict for unseen data
unseen_predictions = model.predict(X)

# unseen features predicted grades
unseen_features['PredictedGrade'] = le.inverse_transform(unseen_predictions)

# print predicted greades
print(unseen_features[['StudentId', 'PredictedGrade']])


KeyError: "Column(s) ['Action', 'Type'] do not exist"